In [119]:
import os
import sys
import random
import numpy as np
import pandas as pd
import pickle as pkl
import scipy.sparse as sp
from Bio import SeqIO
from IPython.display import display
import statistics as st

CELL_LINE = 'GM12878'

In [120]:
def getPairs(cell_line):
    """
        If your cell line is not available in TargetFinder repo,
        Place your ep_pairs.csv file manually under your cell line directory.
    """
    available_cell_lines = ['GM12878', 'HUVEC', 'HeLa-S3', 'IMR90', 'K562', 'NHEK', 'combined']

    if cell_line not in available_cell_lines:
        print('{} cell line is not in available.\nSelect one of {}\n' \
              'Or manually create gcn/data/{}/ep_pairs.csv'.format(cell_line, available_cell_lines, cell_line))
        return None

    if os.path.isfile('gcn/data/{}/ep_pairs.csv'.format(cell_line)):
        print('Reading pairs from local file...')
        ep_pairs = pd.read_csv('gcn/data/{}/ep_pairs.csv'.format(cell_line))
    else:
        print('Reading pairs from remote github repo...')
        ep_pairs = pd.read_csv('https://raw.githubusercontent.com/shwhalen/' \
                               'targetfinder/master/paper/targetfinder/{}/' \
                               'output-ep/pairs.csv'.format(cell_line))
        if not os.path.isdir('gcn/data/{}'.format(cell_line)):
            print('Creating directory for {} cell line...'.format(cell_line))
            os.makedirs('gcn/data/{}'.format(cell_line))
        print('Writing pairs to data/{}/ep_pairs.csv'.format(cell_line))
        ep_pairs.to_csv('gcn/data/{}/ep_pairs.csv'.format(cell_line), index=False)
    return ep_pairs

In [121]:
def getSequences(ep_pairs, hg):
    RefSeqIDs = []

    for k in hg37.keys():
        if k.startswith('NC_0000'):
            RefSeqIDs.append(hg37[k].id)

    chromosomes = ['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', \
               'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', \
               'chr18', 'chr19', 'chr20', 'chr21', 'chr22', 'chrX', 'chrY']

    RefSeqDict = {chromosomes[i]: RefSeqIDs[i] for i in range(len(chromosomes))}

    enhancer_sequences = []
    promoter_sequences = []
    n = len(ep_pairs)

    print('Getting DNA sequences for {} EP pairs...'.format(n))

    for i in range(n):
        enhancer_seq_id = ep_pairs['enhancer_chrom'][i]
        enhancer_seq_start = ep_pairs['enhancer_start'][i] - 1
        enhancer_seq_end = ep_pairs['enhancer_end'][i]

        promoter_seq_id = ep_pairs['promoter_chrom'][i]
        promoter_seq_start = ep_pairs['promoter_start'][i] - 1
        promoter_seq_end = ep_pairs['promoter_end'][i]
        
        enhancer_sequences.append(str(hg37[RefSeqDict[enhancer_seq_id]]
                                    .seq[enhancer_seq_start:enhancer_seq_end]).upper())

        promoter_sequences.append(str(hg37[RefSeqDict[promoter_seq_id]]
                                    .seq[promoter_seq_start:promoter_seq_end]).upper())

    ep_sequences = pd.DataFrame({'enhancer_name': ep_pairs['enhancer_name'][0:n],
                                 'promoter_name': ep_pairs['promoter_name'][0:n],
                                 'enhancer_seq': enhancer_sequences,
                                 'promoter_seq': promoter_sequences})
    return ep_sequences

In [122]:
# DOWNLOAD HUMAN GENOME v37 (3.2 Gb)
# Older version but compatible with genomic coordinates of TargetFinder dataset
# https://www.ncbi.nlm.nih.gov/projects/genome/guide/human/index.shtml
# https://github.com/shwhalen/targetfinder/tree/master/paper/targetfinder

try:
    hg37
except NameError:
    print('Parsing GRCh37 genome...')
    hg37 = SeqIO.to_dict(SeqIO.parse('gcn/data/GRCh37_latest_genomic.fna', 'fasta'))

In [123]:
ep_pairs = getPairs(CELL_LINE)
print('{} EP pairs have been read.'.format(len(ep_pairs)))

ep_pairs = ep_pairs[ep_pairs['label'] == 1].reset_index() # Keep only the interacting pairs
print('{} EP pairs are labeled as 1.'.format(len(ep_pairs)))

ep_sequences = getSequences(ep_pairs, hg37)

Reading pairs from local file...
44313 EP pairs have been read.
2113 EP pairs are labeled as 1.
Getting DNA sequences for 2113 EP pairs...


In [124]:
display(ep_sequences.head())

,enhancer_name,promoter_name,enhancer_seq,promoter_seq
0,GM12878|chr1:9685722-9686400,GM12878|chr1:9747084-9749721,TGACAGGCATGAGCCACCACGCCCGGCAGATTTTTCAAGATATAAT...,TTTTGCCATTTCAAAGAATCTTGGATTTTTCTCTGGGCTCCAGAGA...
1,GM12878|chr1:24136556-24136600,GM12878|chr1:24193468-24194871,GTGGCAACTGAGGCTAAGACCTGGAGCAGGGCAGCTGCTCTCAAG,TGAATTCAAAGTTCAAGAGAAACGAAAACCCGGAAGATGGCTGAGG...
2,GM12878|chr1:24136600-24136932,GM12878|chr1:24193468-24194871,GAAACAGTTGCTACTGTTACCATTCCACCTATCTGGATGCCACAAA...,TGAATTCAAAGTTCAAGAGAAACGAAAACCCGGAAGATGGCTGAGG...
3,GM12878|chr1:24137625-24137875,GM12878|chr1:24193468-24194871,GTGCCAGAGGAGCTGGGGCCAGTACTCCAAAAGGAGACCAAAGACT...,TGAATTCAAAGTTCAAGAGAAACGAAAACCCGGAAGATGGCTGAGG...
4,GM12878|chr1:24139145-24139414,GM12878|chr1:24193468-24194871,GCCCAGAGGCAAGAGTGGAGGCATGTGACAAACAGAAAGAAGTTCC...,TGAATTCAAAGTTCAAGAGAAACGAAAACCCGGAAGATGGCTGAGG...


In [125]:
enh_seqs = []
pro_seqs = []

for i in range(len(ep_sequences)):
    enh_seqs.append(ep_sequences['enhancer_seq'][i])
    pro_seqs.append(ep_sequences['promoter_seq'][i])
    
enh_seqs = list(set(enh_seqs))
pro_seqs = list(set(pro_seqs))

enh_lengths = []
pro_lengths = []

for i in range(len(enh_seqs)):
    enh_lengths.append(len(enh_seqs[i]))

for i in range(len(pro_seqs)):
    pro_lengths.append(len(pro_seqs[i]))

In [126]:
print(len(enh_seqs), "unique enhancers")
print(len(pro_seqs), "unique promoters\n")

print("Enhancer lengths between", min(enh_lengths), '-', max(enh_lengths))
print("Promoter lengths between", min(pro_lengths), '-', max(pro_lengths))

1932 unique enhancers
736 unique promoters

Enhancer lengths between 6 - 6670
Promoter lengths between 119 - 18209


In [127]:
min_index = enh_lengths.index(min(enh_lengths))
print("Shortest enhancer ->", enh_seqs[min_index])

Shortest enhancer -> CTCTGT


In [133]:
print('{:.1f}'.format(st.mean(enh_lengths)), "mean enhancer length")
print('{:.1f}'.format(st.mean(pro_lengths)), "mean promoter length")

print('{:.1f}'.format(st.median(enh_lengths)), "median enhancer length")
print('{:.1f}'.format(st.median(pro_lengths)), "median promoter length")

553.7 mean enhancer length
2633.2 mean promoter length
278.0 median enhancer length
2128.0 median promoter length


In [129]:
print(len(list(filter(lambda length: length >= 200, enh_lengths))), "enhancers with length >= 200")
print(len(list(filter(lambda length: length >= 200, pro_lengths))), "promoters with length >= 200")

1228 enhancers with length >= 200
732 promoters with length >= 200


### FIX THE SEQUENCE LENGTHS

See iEnhancer-2L paper [ https://academic.oup.com/bioinformatics/article/32/3/362/1744331 ]

In [134]:
# Divide enhancers and promoters into 200bp DNA fragments
# Remove fragments with length < 200bp
# Remove fragments with high sequence similarity (cutoff threshold at 80%)

In [162]:
enh_fragments = []
for seq in enh_seqs:
    remained_seq = seq
    while len(remained_seq) >= 200:
        enh_fragments.append(remained_seq[:200])
        remained_seq = remained_seq[200:]

pro_fragments = []
for seq in pro_seqs:
    remained_seq = seq
    while len(remained_seq) >= 200:
        pro_fragments.append(remained_seq[:200])
        remained_seq = remained_seq[200:]

In [165]:
print(len(enh_fragments), "enhancer fragments of 200bp")
print(len(pro_fragments), "promoter fragments of 200bp")

4472 enhancer fragments of 200bp
9319 promoter fragments of 200bp


In [166]:
# CD-HIT or other method to remove highly similar sequences to get rid of redundancy and avoid bias 